In [59]:
from os import path 
import snap
import numpy as np
import pandas as pd
from scipy.stats import bernoulli
from matplotlib import pyplot as plt
from sklearn.cluster import KMeans

In [2]:
%matplotlib inline

#### Test Graphs.

In [3]:
def make_test_G1():
    G = snap.TUNGraph.New()
    for i in xrange(0,8):
        G.AddNode(i)
    G.AddEdge(0,1)
    G.AddEdge(0,2)
    G.AddEdge(0,3)
    G.AddEdge(1,2)
    G.AddEdge(1,5)
    G.AddEdge(2,3)
    G.AddEdge(2,4)
    G.AddEdge(3,4)
    G.AddEdge(4,7)
    G.AddEdge(5,6)
    G.AddEdge(5,7)
    G.AddEdge(6,7)
    return G

In [4]:
def make_test_G2():
    G = snap.TUNGraph.New()
    for i in xrange(0,4):
        G.AddNode(i)
    G.AddEdge(0,1)
    G.AddEdge(1,2)
    G.AddEdge(2,3)
    return G

In [5]:
def make_test_G3():
    G = snap.TUNGraph.New()
    for i in xrange(0,10):
        G.AddNode(i)
    G.AddEdge(0,1)
    G.AddEdge(0,2)
    G.AddEdge(0,3)
    G.AddEdge(0,9)
    G.AddEdge(1,2)
    G.AddEdge(1,3)
    G.AddEdge(2,3)
    G.AddEdge(3,4)
    G.AddEdge(4,5)
    G.AddEdge(4,6)
    G.AddEdge(5,6)
    G.AddEdge(6,7)
    G.AddEdge(7,8)
    G.AddEdge(7,9)
    G.AddEdge(8,9)
    return G

In [6]:
def make_test_G4():
    G = snap.TUNGraph.New()
    G.AddNode(47)
    G.AddNode(82)
    G.AddNode(103)
    G.AddNode(300)
    G.AddEdge(47,82)
    G.AddEdge(47,300)
    G.AddEdge(82,103)
    G.AddEdge(82,300)
    return G

## Spectral Clustering Functions

In [7]:
def get_adjacency_matrix(G):
    '''
    This function might be useful for you to build the adjacency matrix of a
    given graph and return it as a numpy array.
    
    Returns: adjacency matrix A.
    '''
    ##########################################################################
    n = G.GetNodes()
    A = np.zeros((n, n))
    
    for N1 in G.Nodes(): # Iterate over each node.
        for nbr_idx in xrange(N1.GetDeg()): # Iterate over each neighbor of the node.
            N2 = G.GetNI(N1.GetNbrNId(nbr_idx))
            A[N1.GetId(), N2.GetId()] = 1
            A[N2.GetId(), N1.GetId()] = 1

    return A
    ##########################################################################


In [8]:
def test_get_adjacency_matrix():
    G1 = make_test_G1()
    A = get_adjacency_matrix(G1)
    
    # Check that A is what is expected.
    expected_A = np.zeros((8,8))
    expected_A[0,1] = 1
    expected_A[1,0] = 1
    expected_A[0,2] = 1
    expected_A[2,0] = 1
    expected_A[0,3] = 1
    expected_A[3,0] = 1
    expected_A[1,2] = 1
    expected_A[2,1] = 1
    expected_A[1,5] = 1
    expected_A[5,1] = 1
    expected_A[2,3] = 1
    expected_A[3,2] = 1
    expected_A[2,4] = 1
    expected_A[4,2] = 1
    expected_A[3,4] = 1
    expected_A[4,3] = 1
    expected_A[4,7] = 1
    expected_A[7,4] = 1
    expected_A[5,6] = 1
    expected_A[6,5] = 1
    expected_A[5,7] = 1
    expected_A[7,5] = 1
    expected_A[6,7] = 1
    expected_A[7,6] = 1   
    assert(np.array_equal(A, expected_A))
    print("Test passes :)")
    
test_get_adjacency_matrix() 

Test passes :)


In [9]:
def get_sparse_degree_matrix(G):
    '''
    This function might be useful for you to build the degree matrix of a
    given graph and return it as a numpy array. The sparse degree matrix is
    matrix D.
    '''
    ##########################################################################
    n = G.GetNodes()
    D = np.zeros((n, n))
    
    for NI in G.Nodes(): # Iterate over each node.
        D[NI.GetId(), NI.GetId()] = NI.GetDeg()

    return D
    ##########################################################################

In [10]:
def test_get_adjacency_matrix():
    G1 = make_test_G1()
    D = get_sparse_degree_matrix(G1)
    
    # Check that D is what is expected.
    expected_D = np.zeros((8,8))
    expected_D[0,0] = 3
    expected_D[1,1] = 3
    expected_D[2,2] = 4
    expected_D[3,3] = 3
    expected_D[4,4] = 3
    expected_D[5,5] = 3
    expected_D[6,6] = 2
    expected_D[7,7] = 3
    assert(np.array_equal(D, expected_D))
    print("Test passes :)")
    
test_get_adjacency_matrix() 

Test passes :)


Helper functions for $\texttt{normalized_cut_minimization}$.

In [11]:
def compute_D_neg_half(D):
    """
    Given diagonal matrix of degrees D, computes D^{-1/2} matrix.
    """
    diag_D = np.diag(D)
    D_neg_half = np.diag(diag_D**(-0.5))

    return D_neg_half

In [12]:
def test_compute_D_neg_half():
    G1 = make_test_G1()
    D = get_sparse_degree_matrix(G1)
    D_neg_half = compute_D_neg_half(D)
    
    # Check that computed D_neg_half is what is expected.
    expected_D_neg_half = np.zeros((G1.GetNodes(), G1.GetNodes()))
    expected_D_neg_half[0,0] = 1.0 / (3**0.5)
    expected_D_neg_half[1,1] = 1.0 / (3**0.5)
    expected_D_neg_half[2,2] = 1.0 / (4**0.5)
    expected_D_neg_half[3,3] = 1.0 / (3**0.5)
    expected_D_neg_half[4,4] = 1.0 / (3**0.5)    
    expected_D_neg_half[5,5] = 1.0 / (3**0.5)
    expected_D_neg_half[6,6] = 1.0 / (2**0.5)
    expected_D_neg_half[7,7] = 1.0 / (3**0.5)
    assert(np.allclose(D_neg_half, expected_D_neg_half))
    print("Test passes :)")

test_compute_D_neg_half()

Test passes :)


In [13]:
def compute_normalized_graph_Laplacian(A, D, D_neg_half):
    """
    Returns normalized graph Laplacian, D^{-1/2} L D^{1/2}.
    """
    L = D - A # Graph Laplacian.
    normalized_L = np.dot(np.dot(D_neg_half, L), D_neg_half)
    return normalized_L

In [14]:
def test_compute_normalized_graph_Laplacian():
    G2 = make_test_G2()
    A = get_adjacency_matrix(G2)
    D = get_sparse_degree_matrix(G2)
    D_neg_half = compute_D_neg_half(D)
    normalized_L = compute_normalized_graph_Laplacian(A, D, D_neg_half)
    
    # Check that normalized_L is what is expected.
    expected_normalized_L = np.array([[1, -0.70711, 0, 0],
                                      [-0.70711, 1, -0.5, 0],
                                      [0, -0.5, 1, -0.70711],
                                      [0, 0, -0.70711, 1]])  
    assert(np.allclose(normalized_L, expected_normalized_L))
    print("Test passes :)")

test_compute_normalized_graph_Laplacian()

Test passes :)


In [15]:
def get_second_smallest_eigvec(normalized_L):
    """
    Returns the eigenvector associated with the second smallest eigenvalue of the given noramlized
    graph Laplacian.
    """
    eigvals, eigvecs = np.linalg.eigh(normalized_L) # Eigenvalues in ascending order.
    v = eigvecs[:, 1] # Get the eigenvector associated with the second smallest eigenvalue. 
    
    return v

In [16]:
def test_get_second_smallest_eigvec():
    G2 = make_test_G2()
    A = get_adjacency_matrix(G2)
    D = get_sparse_degree_matrix(G2)
    D_neg_half = compute_D_neg_half(D)
    normalized_L = compute_normalized_graph_Laplacian(A, D, D_neg_half)
    v = get_second_smallest_eigvec(normalized_L)
    
    # Check that v is what we expect.
    expected_v = np.array([-0.57735, -0.40825, 0.40824, 0.57735])
    assert(np.allclose(v, expected_v, atol=10**-5))   
    
test_get_second_smallest_eigvec()

In [17]:
def normalized_cut_minimization(G):
    '''
    Implement the normalized cut minimizaton algorithm we derived in the last
    homework here.
    
    Returns set S and set bar_S containing the two found communities.
    '''
    A = get_adjacency_matrix(G)
    D = get_sparse_degree_matrix(G)
    ##########################################################################
    D_neg_half = compute_D_neg_half(D)
    normalized_L = compute_normalized_graph_Laplacian(A, D, D_neg_half)
    v = get_second_smallest_eigvec(normalized_L)
    minimizer_x = np.dot(D_neg_half, v)
    
    # Get cluster assignments.
    S = list(np.where(minimizer_x >= 0)[0])
    neg_S = list(np.where(minimizer_x < 0)[0])
    return S, neg_S

    ##########################################################################

In [18]:
def test_normalized_cut_minimization():
    # ========================= Use G2 to test math is correct. ==============================
    G2 = make_test_G2()
    S, neg_S = normalized_cut_minimization(G2)
    
    # Check that communities are what mathematically we expect.
    expected_S = [2,3]
    expected_neg_S = [0,1]
    assert(S == expected_S)
    assert(neg_S == expected_neg_S)
    
    # =============== Use G1 to check that the communities found make sense. =================
    G1 = make_test_G1()
    S, neg_S = normalized_cut_minimization(G1)
    
    # Check that communites are intuitively what we expect.
    expected_S = [0,1,2,3,4]
    expected_neg_S = [5,6,7]
    assert(S == expected_S)
    assert(neg_S == expected_neg_S)
    
    print("All tests pass!!! :):):)")

test_normalized_cut_minimization()

All tests pass!!! :):):)


In [19]:
def get_modularity(G, community_dict):
    '''
    This function might be useful to compute the modularity of a given cut
    defined by two sets S and neg_S. We would normally require sets S and neg_S
    to be disjoint and to include all nodes in Graph.
    
    - community_dict: maps node id to community
    '''
    ##########################################################################
    A = get_adjacency_matrix(G)
    two_M = float(np.sum(A))
    mod_sum = 0
    for NI in G.Nodes():
        NI_id = NI.GetId()
        for NJ in G.Nodes():
            NJ_id = NJ.GetId()
            if (community_dict[NI_id] == community_dict[NJ_id]):
                mod_sum += A[NI_id, NJ_id] - ((NI.GetDeg() * NJ.GetDeg()) / two_M)
    modularity = mod_sum / two_M
    return modularity
    ##########################################################################


In [20]:
def test_modularity():
    sample_G = snap.LoadEdgeList(snap.PUNGraph, "q1-sample.txt", 0, 1)
    S, neg_S = normalized_cut_minimization(sample_G)
    community_dict = {node_id:0 for node_id in S}
    community_dict.update({node_id:1 for node_id in neg_S})
    modularity = get_modularity(sample_G, community_dict)
    
    # Check that modularity has expected value.
    expected_modularity = 0.09
    epsilon = 10**-3
    assert(abs(modularity - expected_modularity) < epsilon)
    print("Test passes :)")

test_modularity()

Test passes :)


In [32]:
def cluster_by_eig_space(G, k):
    """
    Build reduced space from k smallest eigenvectors so that each node is now represented on
    k numbers, from the k eigenvectors. Then use this reduced representation go cluster nodes.
    Returns dictionary mapping each node id to. Note that it uses the adjacency matrix index
    as the node_id. You have to map this index to the true node_id later.
    """
    print "Cluster with k:", k
    A = get_adjacency_matrix(G)
    D = get_sparse_degree_matrix(G)
    D_neg_half = compute_D_neg_half(D)
    normalized_L = compute_normalized_graph_Laplacian(A, D, D_neg_half)

    # Find eigenvalues.
    eigvals, eigvecs = np.linalg.eigh(normalized_L) # Eigevanlues in ascending ordering.
    v = eigvecs[:, 1:k+1] # The eigenvalues stored in columns. So each row represents a node.
    
    # Cluster with k-means.
    kmeans = KMeans(n_clusters=k, random_state=0).fit(v)
    return {node:cluster for (node, cluster) in enumerate(kmeans.labels_)}

In [33]:
def test_cluster_by_eig_space():
    G3 = make_test_G3()
    
    highest_mod = 0
    highest_mod_k = 0
    highest_mod_comm_dict = None
    for k in xrange(1,8):
        comm_dict = cluster_by_eig_space(G3, k)
        mod = get_modularity(G3, comm_dict)
        if (mod > highest_mod):
            highest_mod = mod
            highest_mod_k = k
            highest_mod_comm_dict = comm_dict
            
    expected_highest_mod_comm_dict = {0:2,1:2,2:2,3:2,4:0,5:0,6:0,7:1,8:1,9:1}
    assert(highest_mod_k == 3)
    assert(highest_mod_comm_dict == expected_highest_mod_comm_dict)
    print "Test passes :)"

test_cluster_by_eig_space()

Cluster with k: 1
Cluster with k: 2
Cluster with k: 3
Cluster with k: 4
Cluster with k: 5
Cluster with k: 6
Cluster with k: 7
Test passes :)


## Cluster Postids Helper Functions

In [23]:
def remap_postids(G):
    """
    Returns a Graph containing all the remapped post_ids so that they go from 0 to n. 
    Also returns the dictionary that maps the ids to their new value.
    """
    postid_map = dict()
    new_G = snap.TUNGraph.New()
    index = 0
    
    # Remap all nodes. Only keep ones with degree > 0.
    for N in G.Nodes():
        if (N.GetDeg() < 1): continue 
        postid_map[N.GetId()] = index
        new_G.AddNode(index)
        index += 1
                
    # Remap all edges.
    for E in G.Edges(): # Edge traversal
        new_G.AddEdge(postid_map[E.GetSrcNId()], postid_map[E.GetDstNId()])
        
    return new_G, postid_map

In [24]:
def test_remap_postids():
    G4 = make_test_G4()
    new_G, postid_map = remap_postids(G4)
    
    # Get nodes in graph. 
    edges = set()
    for E in new_G.Edges():
        edges.add((E.GetSrcNId(), E.GetDstNId()))
    
    expected_postid_map = {47:0,82:1,103:2,300:3}
    expected_edges = {(0,1),(0,3),(1,2),(1,3)}
    assert(postid_map == expected_postid_map)
    assert(edges == expected_edges)

test_remap_postids()

## Time to Actually Cluster the Postids

In [25]:
BASE_PATH = "../data/stats.stackexchange.com/Mixed"
POST_ID_FOLDED_GRAPH_PATH = path.join(BASE_PATH, "Postid_Folded_Graph.graph")
NGRAMID_DICT_PICKLE_PATH = path.join(BASE_PATH, "Bigramid_Dict.pickle")
POSTID_SET_PICKLE_PATH = path.join(BASE_PATH, "Postid_set.pickle")

In [26]:
# Load post_graph.
FIn = snap.TFIn(POST_ID_FOLDED_GRAPH_PATH)
post_graph = snap.TUNGraph.Load(FIn)

In [27]:
print "Original number of nodes in graph:", post_graph.GetNodes()

Original number of nodes in graph: 19725


#### Create remapped post_graph.

In [28]:
eig_post_graph, postid_dict = remap_postids(post_graph)

In [29]:
print "Nodes with degree greater that 0:", eig_post_graph.GetNodes()

Nodes with degree greater that 0: 17139


#### Cluster for each k and compute modularity

In [43]:
community_maps = []
mod_scores = []

In [44]:
for k in xrange(0,250,50):
    if (k == 0): k = 10
    comm_map = cluster_by_eig_space(eig_post_graph, k)
    mod = get_modularity(eig_post_graph, comm_map)
    print "Modularity", mod
    community_maps.append(comm_map)
    mod_scores.append(mod)

Cluster with k: 10
Modularity 7.397733253897822e-05
Cluster with k: 50
Modularity 0.43976184876628965
Cluster with k: 100
Modularity 0.5412240931643605
Cluster with k: 150
Modularity 0.5499615019989026
Cluster with k: 200
Modularity 0.542894315254493


In [45]:
for k in xrange(250,500,50):
    if (k == 0): k = 10
    comm_map = cluster_by_eig_space(eig_post_graph, k)
    mod = get_modularity(eig_post_graph, comm_map)
    print "Modularity", mod
    community_maps.append(comm_map)
    mod_scores.append(mod)

Cluster with k: 250
Modularity 0.537163366470572
Cluster with k: 300
Modularity 0.5266861109597994
Cluster with k: 350
Modularity 0.526417187856714
Cluster with k: 400
Modularity 0.5257398231970715
Cluster with k: 450
Modularity 0.5217433611742635


In [46]:
for k in xrange(500,750,50):
    if (k == 0): k = 10
    comm_map = cluster_by_eig_space(eig_post_graph, k)
    mod = get_modularity(eig_post_graph, comm_map)
    print "Modularity", mod
    community_maps.append(comm_map)
    mod_scores.append(mod)

Cluster with k: 500
Modularity 0.5203227014171681
Cluster with k: 550
Modularity 0.5171775749165793
Cluster with k: 600
Modularity 0.5138722025387538
Cluster with k: 650
Modularity 0.5101391796789309
Cluster with k: 700
Modularity 0.49851099362008344


In [47]:
for k in xrange(750,1001,50):
    if (k == 0): k = 10
    comm_map = cluster_by_eig_space(eig_post_graph, k)
    mod = get_modularity(eig_post_graph, comm_map)
    print "Modularity", mod
    community_maps.append(comm_map)
    mod_scores.append(mod)

Cluster with k: 750
Modularity 0.49600519840901175
Cluster with k: 800
Modularity 0.5015773629178529
Cluster with k: 850
Modularity 0.5018306438572164
Cluster with k: 900
Modularity 0.48268253658044274
Cluster with k: 950
Modularity 0.49793110922234624
Cluster with k: 1000
Modularity 0.49777639389193645


### Look at the Communities for Clustering with the Best k

In [51]:
best_k = 150
best_k_index = 3
best_comm_map = community_maps[best_k_index]
best_mod_score = mod_scores[best_k_index]

### Load the post top words.

In [54]:
BASE_PATH = "../data/stats.stackexchange.com/Mixed"
POST_TOP_NGRAM_PATH = path.join(BASE_PATH, "STATS_20k-Posts_11-top_uni&bigrams_nostem.tsv")

In [56]:
def load_top_ngram_df(topwords_path):
    # Load csv containing the top words.
    posts_df = pd.read_csv(topwords_path, sep = "\t", usecols = 
                           ["Id", "OwnerUserId", "TopWord1", "TopWord2", "TopWord3", "TopWord4", "TopWord5"])
    
    # Clean dataframe.
    posts_df = posts_df.dropna()
    posts_df = posts_df.rename(columns={
            "Id": "post_id", "OwnerUserId": "user_id"})
    posts_df["user_id"] = posts_df["user_id"].astype(np.int64)
    posts_df["post_id"] = posts_df["post_id"].astype(np.int64)
    posts_df = posts_df[posts_df["user_id"] > 0]
    posts_df = posts_df[posts_df["post_id"] > 0]
    

    return posts_df

Add communities column to posts_df.

In [178]:
def add_communities_post_df(post_df, best_comm_map, postid_dict):
    
    # Iterate over rows and add each community to each row.
    community_array = []
    for index, row in post_df.iterrows():
        user_id = row["post_id"]
        A_id = postid_dict.get(user_id, -1)
        if (A_id < 0):
            community_array.append(-1)
        else:
            community_array.append(best_comm_map[A_id])
    post_df.loc[:,'Community'] = community_array
    
    return post_df

In [179]:
post_df_w_comm = add_communities_post_df(post_df, best_comm_map, postid_dict)

Print community sizes.

In [187]:
communities = set(post_df_w_comm["Community"])
for comm in communities:
    print "Community:", comm, "Size:", len(post_df[post_df_w_comm["Community"] == comm])

Community: 0 Size: 87
Community: 1 Size: 8343
Community: 2 Size: 2
Community: 3 Size: 2
Community: 4 Size: 2
Community: 5 Size: 2
Community: 6 Size: 2
Community: 7 Size: 2
Community: 8 Size: 2
Community: 9 Size: 2
Community: 10 Size: 2
Community: 11 Size: 2
Community: 12 Size: 2
Community: 13 Size: 2
Community: 14 Size: 2
Community: 15 Size: 3
Community: 16 Size: 169
Community: 17 Size: 200
Community: 18 Size: 8
Community: 19 Size: 7
Community: 20 Size: 2
Community: 21 Size: 2
Community: 22 Size: 2
Community: 23 Size: 4
Community: 24 Size: 2
Community: 25 Size: 2
Community: 26 Size: 5
Community: 27 Size: 2
Community: 28 Size: 2
Community: 29 Size: 2
Community: 30 Size: 29
Community: 31 Size: 2
Community: 32 Size: 4
Community: 33 Size: 5
Community: 34 Size: 2
Community: 35 Size: 3
Community: 36 Size: 2
Community: 37 Size: 2
Community: 38 Size: 6
Community: 39 Size: 2
Community: 40 Size: 2
Community: 41 Size: 2
Community: 42 Size: 109
Community: 43 Size: 65
Community: 44 Size: 2
Communit

In [188]:
post_df[post_df_w_comm["Community"] == 86]

,post_id,user_id,TopWord1,TopWord2,TopWord3,TopWord4,TopWord5,Community
67,317788,187855,th level,effect th,factor,level factor,levene,86
526,318375,122192,factor,variables clustering,clustering,dataframe,numerical,86
990,318976,164061,alias,column,factors,factor,dummy,86
1417,319526,85650,loadings,sign,varimax,factor,sign factor,86
2466,320895,26456,subjects subjects,dfs,subjects,factor,returns,86
2930,321472,129390,communalities,efa,factor,eigenvalue,accounted variance,86
3252,321886,187640,irt,cfa,factor cfa,factor,single factor,86
3672,322430,8013,analysis,complete factor,complete,factor,consider complete,86
5036,324173,192279,dk,wrong dk,factor analysis,scale wrong,factor,86
5116,324282,26569,portfolio,portfolio factor,averaged time,fama,factor,86


In [191]:
post_df[post_df_w_comm["Community"] == 16]

,post_id,user_id,TopWord1,TopWord2,TopWord3,TopWord4,TopWord5,Community
180,317930,2392,model,mis specified,mis,model mis,adding parameters,16
249,318012,188026,model residuals,model,plots,appear bit,appears outliers,16
429,318256,188195,error variable,predict outcome,model,outcome,model diagram,16
450,318282,1390,smooth,effect,smooth effect,smooth interaction,model,16
498,318340,31363,deviance,model,deviance decrease,expect deviance,added model,16
499,318341,138414,crimes,house prices,house,prices,model,16
653,318537,188357,chemotherapy,chemotherapy variable,variable equals,cox model,model,16
683,318576,86652,parameter,model selection,model,uncertainty,maximum likelihood,16
749,318664,178215,replicated,performance model,model,able train,performance,16
856,318796,113090,statistic,define conditional,conditional model,model,measurable,16


TODO
- word to vec
- average top five words in post, compute similarity in cluster